<a href="https://colab.research.google.com/github/larasauser/master/blob/main/visu4panneaux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from pyproj import Transformer, CRS

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def format_coord_deg(val):
    return f'{val:.3f}°'

def add_coords_wgs84(ax, meta, show_y=True):
    transform = meta['transform']
    nrows, ncols = meta['height'], meta['width']

    x_idx = np.linspace(0, ncols - 1, 5, dtype=int)
    y_idx = np.linspace(0, nrows - 1, 5, dtype=int)

    x_pix = transform[2] + x_idx * transform[0]
    y_pix = transform[5] + y_idx * transform[4]

    raster_crs = CRS(meta['crs'])
    if raster_crs.to_epsg() != 4326:
        transformer = Transformer.from_crs(raster_crs, "EPSG:4326", always_xy=True)
        x_pix, y_pix = transformer.transform(x_pix, y_pix)

    ax.set_xticks(x_idx)
    ax.set_xticklabels([format_coord_deg(x) for x in x_pix])

    if show_y:
        ax.set_yticks(y_idx)
        ax.set_yticklabels([format_coord_deg(y) for y in y_pix])
    else:
        ax.set_yticks([])


In [4]:
def read_raster(path):
    with rasterio.open(path) as src:
        img = src.read(1).astype(np.float32)
        meta = src.meta.copy()

    img[img == -9999] = np.nan
    return img, meta


In [8]:
def visu_5_fidele(
    gt_img,
    target_img,
    filled_img,
    ref_img,
    gap_mask,
    gt_path,
    target_path,
    ref_path,
    band=0,
    save=False,
    output_dir=None,
    filename="visu_ndvi.png",
    err_vmin=None,
    err_vmax=None
):
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import numpy as np
    import rasterio
    import os

    # --- Ensure 3D ---
    if gt_img.ndim == 2:
        gt_img = gt_img[:, :, np.newaxis]
    if target_img.ndim == 2:
        target_img = target_img[:, :, np.newaxis]
    if filled_img.ndim == 2:
        filled_img = filled_img[:, :, np.newaxis]
    if ref_img.ndim == 2:
        ref_img = ref_img[:, :, np.newaxis]

    # --- Extract names ---
    gt_name = os.path.splitext(os.path.basename(gt_path))[0]
    target_name = os.path.splitext(os.path.basename(target_path))[0]
    ref_name = os.path.splitext(os.path.basename(ref_path))[0]

    # --- Replace -9999 by NaN ---
    gt_band = np.where(gt_img[:, :, band] == -9999, np.nan, gt_img[:, :, band])
    target_band = np.where(target_img[:, :, band] == -9999, np.nan, target_img[:, :, band])
    filled_band = np.where(filled_img[:, :, band] == -9999, np.nan, filled_img[:, :, band])
    ref_band = np.where(ref_img[:, :, band] == -9999, np.nan, ref_img[:, :, band])

    # --- Error only on gaps ---
    error_band = np.full_like(gt_band, np.nan)
    error_band[gap_mask] = gt_band[gap_mask] - filled_band[gap_mask]

    # --- Extent from GT ---
    with rasterio.open(gt_path) as src:
        bounds = src.bounds
    extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

    # --- Colormaps ---
    ndvi_cmap = cm.get_cmap("gray").copy()
    ndvi_cmap.set_bad(color="red")

    # --- Figure ---
    fig, axes = plt.subplots(1, 5, figsize=(30, 6), constrained_layout=True)
    axes = axes.flatten()

    # --- Panels ---
    im1 = axes[0].imshow(target_band, cmap=ndvi_cmap, vmin=-1, vmax=1, extent=extent)
    axes[0].set_title(f"GAP - {target_name}", fontsize=18, fontweight="bold")
    axes[0].set_xlabel("Longitude", fontsize=16, fontweight="bold")
    axes[0].set_ylabel("Latitude", fontsize=16, fontweight="bold")

    im0 = axes[1].imshow(gt_band, cmap=ndvi_cmap, vmin=-1, vmax=1, extent=extent)
    axes[1].set_title(f"GT - {gt_name}", fontsize=18, fontweight="bold")
    axes[1].set_xlabel("Longitude", fontsize=16, fontweight="bold")

    im2 = axes[2].imshow(filled_band, cmap=ndvi_cmap, vmin=-1, vmax=1, extent=extent)
    axes[2].set_title("RE", fontsize=18, fontweight="bold")
    axes[2].set_xlabel("Longitude", fontsize=16, fontweight="bold")

    im3 = axes[3].imshow(ref_band, cmap=ndvi_cmap, vmin=-1, vmax=1, extent=extent)
    axes[3].set_title(f"REF - {ref_name}", fontsize=18, fontweight="bold")
    axes[3].set_xlabel("Longitude", fontsize=16, fontweight="bold")

    im4 = axes[4].imshow(
        error_band,
        cmap=cm.RdYlGn,
        vmin=err_vmin,
        vmax=err_vmax,
        extent=extent
    )
    axes[4].set_title("ERR (GT - RE)", fontsize=18, fontweight="bold")
    axes[4].set_xlabel("Longitude", fontsize=16, fontweight="bold")

    # --- Colorbars ---
    cbar = fig.colorbar(im2, ax=axes[:4], orientation="vertical",
                        fraction=0.03, pad=0.01)
    cbar.set_label("NDVI", fontsize=16)
    for t in cbar.ax.get_yticklabels():
        t.set_fontsize(13)

    cbar2 = fig.colorbar(im4, ax=axes[4], orientation="vertical",
                         fraction=0.03, pad=0.01)
    cbar2.set_label("NDVI Error", fontsize=16)
    for t in cbar2.ax.get_yticklabels():
        t.set_fontsize(13)

    # --- Ticks ---
    for ax in axes:
        ax.tick_params(axis="x", labelsize=14)
        ax.tick_params(axis="y", labelsize=14)
        ax.yaxis.offsetText.set_fontsize(14)

    # --- Show first ---
    plt.show()

    # --- Optional save ---
    if save:
        if output_dir is None:
            raise ValueError("output_dir doit être défini si save=True")

        os.makedirs(output_dir, exist_ok=True)
        out_path = os.path.join(output_dir, filename)
        fig.savefig(out_path, dpi=300, bbox_inches="tight")
        print(f"✔ Figure sauvegardée : {out_path}")

    return fig


In [10]:
# =========================
# PATHS – NDVI IMAGES
# =========================

gap_path = "/content/drive/MyDrive/NDVI_grancy_Landsat8_10/imputed/holes_real/NDVI_2023-11-08.tif"

gt_path = "/content/drive/MyDrive/NDVI_grancy_Landsat8_10/imputed/NDVI_2023-11-08.tif"

filled_path = "/content/drive/MyDrive/NDVI_grancy_Landsat8_10/imputed/holes_real/filled_nspi3/NDVI_2023-11-08.tif"

ref_path = "/content/drive/MyDrive/NDVI_grancy_Landsat8_10/imputed/NDVI_2023-10-07.tif"

output_dir = "/content/drive/MyDrive/NDVI_grancy_Landsat8_10/imputed/holes_real/filled_nspi3/graph_1"


In [ ]:
# Replace -9999 by NaN
gt_ = np.where(gt_img[:, :, band] == -9999, np.nan, gt_img[:, :, band])
target_band = np.where(target_img[:, :, band] == -9999, np.nan, target_img[:, :, band])
filled_band = np.where(filled_img[:, :, band] == -9999, np.nan, filled_img[:, :, band])
ref_band = np.where(ref_img[:, :, band] == -9999, np.nan, ref_img[:, :, band])

# --- GAP MASK : based ONLY on NaNs in target image ---
gap_mask = np.isnan(target_band)


In [13]:
fig = visu_5_fidele(
    gt_img=gt_path,
    target_img=gap_path,
    filled_img=filled_path,
    ref_img=ref_path,
    gap_mask=gap_mask,
    gt_path=gt_path,
    target_path=gap_path,
    ref_path=ref_path,
    err_vmin=-0.2,
    err_vmax=0.2,
    save=False
)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''